# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [21]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

We consider the **standard OLG-model** where:

We assume that utility is given by $u(c)$ with constant relative risk aversion (CRRA utility function): 

\begin{equation}
  u(c)=\begin{cases}
    \frac{c^{1-\sigma}-1}{1-\sigma} , & \sigma ≠ 1 \\
    log c, & \sigma = 1
  \end{cases}
\end{equation}

Individuals live for two periods, t = 1,2. $L_t$ individuals are born in period t and we assume that the population grows with a constant rate, n: 

$$ L_t = L_{t-1}(1+n) $$

agents derive utility while alive: 

$$ U_t = u(c_{1t})+\frac{1}{1+\rho} u(c_{2t}) $$

Agtens' budget constraint in each period is given by: 

$$ c_{1t} + s_t = w_t $$

$$ c_{2t+1} = (1+r_{t+1}) s_t $$

This implies that agents' lifetime budget constraint is given by: 

$$ c_{1t} + \frac{c_{2t+1}}{1+r_{t+1}} = w_t $$



# Steady state

## Analytical solution

Every agtens born at time t maximize utility subject to their lifetime budget constraint:

First we define all **symbols**:

In [52]:
c_1t = sm.symbols('c_1t')
c_2t = sm.symbols('c_2t+1')
rho = sm.symbols('rho')
u_1 = sm.symbols('u_1')
u_2 = sm.symbols('u_2')
sigma = sm.symbols('sigma')
w = sm.symbols('w_t')
r_t1 = sm.symbols('r_t+1')
lamb = sm.symbols('lambda')

In [46]:
# Define equations
budget_c = sm.Eq(c_1t + c_2t/(1+r_t1), w)

u_c1 = ((c_1t)**(1-sigma)-1)/(1-sigma)
u_c2 = ((c_2t)**(1-sigma)-1)/(1-sigma)

U = u_c1 + u_c2/(1+rho)
U

(c_1t**(1 - sigma) - 1)/(1 - sigma) + (c_2t+1**(1 - sigma) - 1)/((1 - sigma)*(rho + 1))

In [45]:
#solve the budget constraint for $c_{1t}$
budget_c1 = sm.solve(budget_c,c_1t)[0]
budget_c1

(-c_2t+1 + r_t+1*w_t + w_t)/(r_t+1 + 1)

In [48]:
# We substitute budget_c1 into U_lifetime: 

U_lifetime = U.subs(c_1t, budget_c1)
U_lifetime

(c_2t+1**(1 - sigma) - 1)/((1 - sigma)*(rho + 1)) + (((-c_2t+1 + r_t+1*w_t + w_t)/(r_t+1 + 1))**(1 - sigma) - 1)/(1 - sigma)

In [51]:
# Maximize lifetime utility

max_c2 = sm.diff(U_lifetime, c_2t)
max_c2 = sm.simplify(max_c2)
max_c2

(-c_2t+1*((-c_2t+1 + r_t+1*w_t + w_t)/(r_t+1 + 1))**(1 - sigma)*(rho + 1) + c_2t+1**(1 - sigma)*(-c_2t+1 + r_t+1*w_t + w_t))/(c_2t+1*(rho + 1)*(-c_2t+1 + r_t+1*w_t + w_t))

In [58]:
# Lagrangian
budget_l = c_1t + c_2t/(1+r_t1) - w
L = U - lamb * budget_l
L



-lambda*(c_1t + c_2t+1/(r_t+1 + 1) - w_t) + (c_1t**(1 - sigma) - 1)/(1 - sigma) + (c_2t+1**(1 - sigma) - 1)/((1 - sigma)*(rho + 1))

In [59]:
L_c1 = sm.diff(L, c_1t)
L_c1

-lambda + c_1t**(1 - sigma)/c_1t

In [84]:
L_c2 = sm.diff(L,c_2t)
L_c2

-lambda/(r_t+1 + 1) + c_2t+1**(1 - sigma)/(c_2t+1*(rho + 1))

In [70]:
L_l = sm.diff(L, lamb)
L_l

-c_1t - c_2t+1/(r_t+1 + 1) + w_t

In [85]:
L_c2_eq = sm.Eq(L_c2,0)
L_c2_eq



Eq(-lambda/(r_t+1 + 1) + c_2t+1**(1 - sigma)/(c_2t+1*(rho + 1)), 0)

In [89]:
L_c2_s = sm.solve(L_c2_eq, lamb)
L_c2_s

[c_2t+1**(-sigma)*(r_t+1 + 1)/(rho + 1)]

For later use, we turn the solution into a **Python funciton**

In [5]:
ss_func = sm.lambdify((s,g,n,alpha,delta),kss)

## Numerical solution

We can re-write the equation for the steady state capital per capita as

\\[ 0 = \frac{1}{(1+n)(1+g)}[sf(\tilde{k}^{\ast})+(1-\delta)\tilde{k}^{\ast}] - \tilde{k}^{\ast} \\]

whereby it clearly becomes a **root-finding problem**. Such a problem can be solved by a **bisection method**.

Define the model **parameters**:

In [6]:
s = 0.2
g = 0.02
n = 0.01
alpha = 1/3
delta = 0.1

**Solve numerically** for the steady state:

In [7]:
solution = modelproject.solve_for_ss(s,g,n,alpha,delta)

print(f'analytical solution is: {ss_func(s,g,n,alpha,delta):.3f}')
print(f' numerical solution is: {solution.root:.3f}')

analytical solution is: 1.904
 numerical solution is: 1.904


# Further analysis

ADD FURTHER ANALYSIS, VISUALIZATIONS AND EXTENSIONS.

# Conclusion

ADD CONCISE CONCLUSION.